# Classify Intent

In [81]:
import unicodedata
import pandas as pd
from string import punctuation
from nltk.stem import RSLPStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

clf = MultinomialNB()
stemmer = RSLPStemmer()
vectorizer = TfidfVectorizer()

In [115]:
df = pd.DataFrame([
     # sales
     ['vendas do último mês', 'sales'],
     ['quanto vendi esse mês', 'sales'],
     ['quanto vendi no mês de agosto', 'sales'],
     ['quais são os 10 produtos que mais vendi esse ano', 'sales'],
     ['quais são os 10 produtos que mais vendi esse mês', 'sales'],
     ['quais são os 10 produtos que mais vendi hoje', 'sales'],
     ['qual produto eu mais vendi hoje', 'sales'],
     # products
     ['quantos produtos eu tenho na minha base', 'products'],
     ['quantos produtos eu tenho na minha base de dados', 'products'],
     ['quantos produtos eu tenho na minha loja', 'products'],
     ['quantos refrigerantes eu vendo na minha loja', 'products'],
     ['quanto é o preço da coca-cola', 'products'],
     ['qual é o preço da coca-cola', 'products'],
     ['quantos refrigerantes tenho no estoque', 'products']
 ], columns=['message', 'label'])

'''
 PREPROCESSING TEXT
'''

def normalize(message):
    normalized = unicodedata.normalize('NFKD', message).encode('ASCII', 'ignore').decode('utf-8')
    return normalized

def stem(message):
    words = word_tokenize(message)
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)

def remove_stopwords(message):
    blacklist = set(stopwords.words('portuguese') + list(punctuation))
    clean_words = [word for word in word_tokenize(message) if word not in blacklist]
    return ' '.join(clean_words)

def preprocess_message(message):
    message = normalize(message)
    message = remove_stopwords(message)
#     message = stem(message)
    return message

def preprocess(df, column_in='message', column_out='message_clean'):
    df[column_out] = df[column_in].apply(lambda message : preprocess_message(message))

'''
 VECTORIZE
'''
def vectorize(df, column = 'message'):
    X = vectorizer.fit_transform(df[column])
    return X

def vectorize_message(message):
    return vectorizer.transform([message])

'''
 PREDICTION
'''
def predict(model, message):
    vector = vectorize_message(message)
    predict_label = model.predict(vector)
    predict_proba = model.predict_proba(vector)
    return { 'label': predict_label, 'probability': predict_proba, 'classes': model.classes_ }

In [116]:
preprocess(df, column_in='message', column_out='message_clean')
X = vectorize(df, column='message_clean')

print(vectorizer.get_feature_names())
print('corpus shape: ', X.shape)

['10', 'agosto', 'ano', 'base', 'coca', 'cola', 'dados', 'estoque', 'hoje', 'loja', 'mes', 'preco', 'produto', 'produtos', 'quais', 'quanto', 'quantos', 'refrigerantes', 'sao', 'ultimo', 'vendas', 'vendi', 'vendo']
corpus shape:  (14, 23)


In [117]:
model = clf.fit(X, df['label'])

In [118]:
predict(model, 'quantas coca-colas tenho no estoque?')

{'label': array(['products'], dtype='<U8'),
 'probability': array([[0.71838278, 0.28161722]]),
 'classes': array(['products', 'sales'], dtype='<U8')}